In [195]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import PCA

import transformers
from transformers import AutoTokenizer, AutoModel, BertConfig
from transformers import AdamW
from transformers import get_scheduler
from transformers import get_linear_schedule_with_warmup

In [85]:
embeddings = torch.load('/content/card2vec_WOE_emb-300_epochs-20_lr-0.001_bs-256.pt')
embeddings.cpu()

tensor([[-0.1856,  0.6152, -0.1985,  ...,  0.9244, -1.1932, -0.1201],
        [-0.3628, -0.1604, -1.2523,  ...,  0.7354, -0.2620, -0.4454],
        [ 0.4970, -0.1589, -1.1306,  ..., -1.5528, -2.7197,  1.9941],
        ...,
        [ 0.0845,  0.9935, -1.8502,  ...,  0.8019,  1.5886, -0.8511],
        [-0.6594, -0.8515,  1.1217,  ..., -1.2245,  0.2304, -1.2802],
        [ 1.3041, -0.8184, -0.5291,  ..., -0.0234, -1.3057, -1.6079]])

In [86]:
import pandas as pd
card_indices = pd.read_pickle(r'card_names_WOE.pkl')[1]

In [ ]:
embeddings.shape

torch.Size([329, 300])

In [87]:
text_data = pd.read_pickle(r'oracle_cards.pkl')

In [88]:
card_names = list(card_indices.values())
card_names = [i.strip() for i in card_names]
len(card_names)

329

In [89]:
set_data = []

for elem in card_names:
  for t in text_data:
    if elem in t['name']:
      set_data.append(t)
      break

In [193]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert = AutoModel.from_pretrained("bert-base-uncased")


TypeError: ignored

In [194]:
sentences = [s['oracle_text'] for s in set_data]

null_indices = []
for i, s in enumerate(sentences):
  if s is None:
    null_indices.append(i)

sentences = [s for i, s in enumerate(sentences) if i not in null_indices]

In [92]:
max_len = 0

# For every sentence...
for s in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    if s is not None:
     input_ids = tokenizer.encode(s, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  131


In [93]:
indices_to_keep = [i for i in range(embeddings.shape[0]) if i not in null_indices] # List of indices that should be kept
y = embeddings[torch.LongTensor(indices_to_keep)]

In [94]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original:  Enchanted creatures you control get +2/+2.
Token IDs: tensor([  101, 22454,  7329,  2017,  2491,  2131,  1009,  1016,  1013,  1009,
         1016,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [95]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, y)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset) + 1)
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  196 training samples
   48 validation samples


In [96]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [98]:
class CustomBert(transformers.PreTrainedModel):
    '''Custom model class
       ------------------
       Now the trick is not to inherit the class from `nn.Module` but `transformers.PretrainedModel`
       Also you need to pass the model config during initialisation'''

    def __init__(self, bert):
        super(CustomBert, self).__init__(config=BertConfig.from_pretrained('bert-base-uncased'))
        self.bert = bert

        self.l1 = nn.Linear(768, 768)

        self.mha = nn.MultiheadAttention(embed_dim=768, num_heads=12, dropout=.25)

        self.l2 = nn.Linear(768, 300)

        self.ln = nn.LayerNorm(768)

        self.do = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, sent_id, mask):
        '''For simplicity I have added only one linear layer, you can create any type of network you want'''

        bert_out = self.bert(sent_id, attention_mask=mask)
        o = bert_out.last_hidden_state[:,0,:]
        o = self.ln(o + self.mha(o,o,o)[0])
        #o = self.ln(o + self.do(self.relu(self.l1(o))))
        o = self.l2(o)
        return o

In [111]:
model = CustomBert(bert)

model.cuda()

CustomBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [114]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [115]:

# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 100

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 2, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [116]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [117]:
training_stats = []

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}. '.format(step, len(train_dataloader)))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.

        loss = torch.mean(1 - nn.CosineSimilarity(dim=1)(model(b_input_ids, b_input_mask), b_labels))

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += torch.sum(loss).item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")


    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            loss = torch.mean(1 - nn.CosineSimilarity(dim=1)(model(b_input_ids, b_input_mask), b_labels))


        # Accumulate the validation loss.
        total_eval_loss += loss.item()


    # Report the final accuracy for this validation run.


    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
        }
    )

print("")
print("Training complete!")


======== Epoch 1 / 100 ========
Training...

  Average training loss: 0.97

Running Validation...
  Validation Loss: 0.83

======== Epoch 2 / 100 ========
Training...

  Average training loss: 0.81

Running Validation...
  Validation Loss: 0.74

======== Epoch 3 / 100 ========
Training...

  Average training loss: 0.77

Running Validation...
  Validation Loss: 0.70

======== Epoch 4 / 100 ========
Training...

  Average training loss: 0.72

Running Validation...
  Validation Loss: 0.69

======== Epoch 5 / 100 ========
Training...


KeyboardInterrupt: ignored

In [ ]:
# cos_sim for any two elements?

In [196]:
## Pairwise testing

In [16]:
y_norm = y / y.norm(dim=1)[:, None]

In [17]:
sims = torch.mm(y_norm, y_norm.t())
sims = torch.triu(sims)
sims = torch.Tensor.repeat(sims.unsqueeze(-1), (1, 1, 128)).unsqueeze(-1)

In [522]:
sims.shape

torch.Size([244, 244, 128, 1])

In [18]:
pairs = []
for i in input_ids:
  for j in input_ids:
    pairs.append(torch.cat([i,j]))
pairs = torch.stack(pairs, dim=0)
pairs = pairs.reshape(244,244, 128)

In [19]:
mask_pairs = []
for i in attention_masks:
  for j in attention_masks:
    mask_pairs.append(torch.cat([i,j]))
mask_pairs = torch.stack(mask_pairs, dim=0)
mask_pairs = mask_pairs.reshape(244,244, 128)
#mask_pairs = mask_pairs.reshape(len(attention_masks), len(attention_masks), -1)

In [22]:
pairs_data = torch.cat([pairs.unsqueeze(-1).to(device), mask_pairs.unsqueeze(-1).to(device), sims.to(device)], -1)

In [23]:
pairs_data = torch.flatten(pairs_data, start_dim=0, end_dim=1)
pairs_data.shape

torch.Size([59536, 128, 3])

In [24]:
keep_indices = torch.unique(torch.nonzero(pairs_data[:,:,2], as_tuple=True)[0])

pairs_data = pairs_data[keep_indices]

In [47]:
sim_scores = pairs_data[:, 0, 2]
input_pairs = pairs_data[:, :, 0].type(torch.IntTensor)
mask_pairs = pairs_data[:, :, 1].type(torch.IntTensor)

In [27]:
input_pairs.shape

torch.Size([29890, 128])

In [501]:
type(pairs)

In [48]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_pairs, mask_pairs, sim_scores)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset) + 1)
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

23,913 training samples
5,977 validation samples


In [49]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [72]:
class CustomBert(transformers.PreTrainedModel):
    '''Custom model class
       ------------------
       Now the trick is not to inherit the class from `nn.Module` but `transformers.PretrainedModel`
       Also you need to pass the model config during initialisation'''

    def __init__(self, bert):
        super(CustomBert, self).__init__(config=BertConfig.from_pretrained('bert-base-uncased'))
        self.bert = bert

        self.l1 = nn.Linear(768, 768)

        self.mha = nn.MultiheadAttention(embed_dim=768, num_heads=8, dropout=.5)

        self.l2 = nn.Linear(768, 1)

        self.ln = nn.LayerNorm(768)

        self.do = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, sent_id, mask):
        '''For simplicity I have added only one linear layer, you can create any type of network you want'''

        bert_out = self.bert(sent_id, attention_mask=mask)
        o = bert_out.last_hidden_state[:,0,:]
        #o = self.ln(o + self.mha(o,o,o)[0])
        o = self.ln(o + self.do(self.relu(self.l1(o))))
        o = self.l2(o)
        return o

In [73]:
model = CustomBert(bert)

model.cuda()

CustomBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]], device='cuda:0')

In [74]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [76]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [77]:
training_stats = []

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}. '.format(step, len(train_dataloader)))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.

        loss = nn.MSELoss()(model(b_input_ids, b_input_mask), b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += torch.sum(loss).item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.

    print("")
    print("  Average training loss: {0:.3f}".format(avg_train_loss))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")


    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            loss = nn.MSELoss()(model(b_input_ids, b_input_mask), b_labels)


        # Accumulate the validation loss.
        total_eval_loss += loss.item()


    # Report the final accuracy for this validation run.


    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.

    print("  Validation Loss: {0:.3f}".format(avg_val_loss))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
        }
    )

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


  Batch    40  of    748. 
  Batch    80  of    748. 
  Batch   120  of    748. 
  Batch   160  of    748. 
  Batch   200  of    748. 
  Batch   240  of    748. 
  Batch   280  of    748. 
  Batch   320  of    748. 
  Batch   360  of    748. 
  Batch   400  of    748. 
  Batch   440  of    748. 
  Batch   480  of    748. 
  Batch   520  of    748. 
  Batch   560  of    748. 
  Batch   600  of    748. 
  Batch   640  of    748. 
  Batch   680  of    748. 
  Batch   720  of    748. 


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



  Average training loss: 0.029

Running Validation...


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([25])) that is different to the input size (torch.Size([25, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


  Validation Loss: 0.020

======== Epoch 2 / 3 ========
Training...
  Batch    40  of    748. 
  Batch    80  of    748. 
  Batch   120  of    748. 
  Batch   160  of    748. 
  Batch   200  of    748. 
  Batch   240  of    748. 
  Batch   280  of    748. 
  Batch   320  of    748. 
  Batch   360  of    748. 
  Batch   400  of    748. 
  Batch   440  of    748. 
  Batch   480  of    748. 
  Batch   520  of    748. 
  Batch   560  of    748. 
  Batch   600  of    748. 
  Batch   640  of    748. 
  Batch   680  of    748. 
  Batch   720  of    748. 

  Average training loss: 0.023

Running Validation...
  Validation Loss: 0.022

======== Epoch 3 / 3 ========
Training...
  Batch    40  of    748. 
  Batch    80  of    748. 
  Batch   120  of    748. 
  Batch   160  of    748. 
  Batch   200  of    748. 
  Batch   240  of    748. 
  Batch   280  of    748. 
  Batch   320  of    748. 
  Batch   360  of    748. 
  Batch   400  of    748. 
  Batch   440  of    748. 
  Batch   480  of    748. 

In [60]:
model.eval()

text_data

[{'name': 'Static Orb',
  'mana_cost': '{3}',
  'colors': [],
  'cmc': 3.0,
  'type_line': 'Artifact',
  'oracle_text': "As long as Static Orb is untapped, players can't untap more than two permanents during their untap steps.",
  'power': None,
  'toughness': None,
  'loyalty': None,
  'defense': None,
  'layout': 'normal',
  'card_faces': None},
 {'name': 'Sensory Deprivation',
  'mana_cost': '{U}',
  'colors': ['U'],
  'cmc': 1.0,
  'type_line': 'Enchantment — Aura',
  'oracle_text': 'Enchant creature\nEnchanted creature gets -3/-0.',
  'power': None,
  'toughness': None,
  'loyalty': None,
  'defense': None,
  'layout': 'normal',
  'card_faces': None},
 {'name': 'Road of Return',
  'mana_cost': '{G}{G}',
  'colors': ['G'],
  'cmc': 2.0,
  'type_line': 'Sorcery',
  'oracle_text': 'Choose one —\n• Return target permanent card from your graveyard to your hand.\n• Put your commander into your hand from the command zone.\nEntwine {2} (Choose both if you pay the entwine cost.)',
  'power

In [71]:
help(model.forward())

TypeError: ignored